In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [12]:
train_dataset = dsets.MNIST(root = "./data",
                           train=True,
                           transform=transforms.ToTensor(),
                           download=True)
len(train_dataset)

60000

In [10]:
train_dataset[0][0].numpy().shape

(1, 28, 28)

In [13]:
test_dataset = dsets.MNIST(root = "./data",
                           train=False,
                           transform=transforms.ToTensor())

* totaldata:60000
* minibatch:100
    iterations:3000
       1 iteration: one mini-batch f/b pass
* epochs
    1 epoch: running through whole dataset
       epochs = iterations/ (totaldata/minibatch) = 3000 / 600 = 5

In [17]:
totaldata = len(train_dataset)
batch_size = 100
n_iters = 3000
num_epochs = int(n_iters/(totaldata/batch_size))

In [19]:
train_loader = torch.utils.data.DataLoader(dataset= train_dataset,
                                          batch_size= batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset= test_dataset,
                                          batch_size= batch_size,
                                          shuffle=False)

In [44]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
    
    def forward(self, x):
        out = self.linear(x)
        return out

input_dim = 28* 28
output_dim = 10
model = LogisticRegressionModel(input_dim, output_dim)

In [45]:
criterion  = nn.CrossEntropyLoss()
eta = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr = eta)

In [47]:
print(list(model.parameters()))

[Parameter containing:
tensor([[-0.0067,  0.0197, -0.0163,  ..., -0.0215, -0.0014, -0.0150],
        [-0.0140,  0.0160,  0.0049,  ..., -0.0150, -0.0188,  0.0011],
        [-0.0220, -0.0129,  0.0350,  ...,  0.0338,  0.0030,  0.0311],
        ...,
        [ 0.0309,  0.0057, -0.0320,  ...,  0.0124, -0.0008, -0.0200],
        [ 0.0142,  0.0310,  0.0294,  ..., -0.0254, -0.0025,  0.0236],
        [-0.0033, -0.0192, -0.0080,  ..., -0.0170,  0.0130,  0.0155]],
       requires_grad=True), Parameter containing:
tensor([ 2.1990e-02,  3.1188e-02, -2.5442e-03, -4.0659e-03, -6.0600e-03,
        -4.2688e-05,  3.0836e-02,  8.1216e-03,  2.9412e-03,  3.4640e-02],
       requires_grad=True)]


In [53]:
iter = 0 
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # kload images as variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # clear gradients
        optimizer.zero_grad()
        
        # forward 
        outpus = model(images)
        
        # calcualte loss
        loss = criterion(outpus, labels)
        
        # backward gradients
        loss.backward()
        
        # updateing parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            correct = 0
            total =0 
            for images, labels in test_loader:
                 # kload images as variable
                images = Variable(images.view(-1, 28*28))
        
                # forward 
                outputs = model(images)
                
                _, predicted = torch.max(outputs.data, 1)

                # backward gradients
                total += labels.size(0)

                # updateing parameters
                correct += (predicted == labels).sum()
            
            accuracy = 100 * correct / total
            
            print(f"iter {iter}, loss {loss}, accuracy {accuracy}")

iter 500, loss 1.600559949874878, accuracy 75
iter 1000, loss 1.3492779731750488, accuracy 78
iter 1500, loss 1.2272416353225708, accuracy 80
iter 2000, loss 1.113019585609436, accuracy 81
iter 2500, loss 0.9918678998947144, accuracy 82
iter 3000, loss 0.9861036539077759, accuracy 83
